## [준비]

### Load libraries

In [1]:
import re
import json
import psycopg2
import numpy as np
import pandas as pd
import pandas.io.sql as psql

### Connect to DB server

In [2]:
with open('DBMS_config.json', 'r') as fp:
    dbms_params = json.load(fp)
    conn = psycopg2.connect(**dbms_params)

## [문제 1]

### 문제 1
- visit_occurrence 테이블은 병원에 방문한 환자들의 방문식별번호(id), 병원 방문
시작일자, 종료일자, 방문 타입(내원, 외래 등) 등 병원 방문과 관련된 정보를
포함하고 있습니다. 내원일수는 환자가 요양기관을 방문하여 진료를 받은 일수이며,
`내원일수 = 방문종료일자 - 방문시작일자 + 1` 으로 계산합니다. 모든 환자에 대해
총 내원일수를 구하고 총 내원일수의 최대값과 총 내원일수 최대값을 가지는
환자수를 찾는 쿼리를 작성합니다.

    a. 방문시작일자는 visit_start_date, 방문종료일자는 visit_end_date 를 사용합니다.

### 문제 1 답안

In [3]:
query = """
WITH TBL AS (
	SELECT
	     *,
	     MIN(TOT_VISIT_DAYS) OVER(),
	     MAX(TOT_VISIT_DAYS) OVER()
	FROM (
		SELECT
			PERSON_ID,
			SUM(VISIT_DAYS) AS TOT_VISIT_DAYS
		FROM
			(
			SELECT
				PERSON_ID,
				(DATE(VISIT_END_DATE) - DATE(VISIT_START_DATE) + 1) AS VISIT_DAYS
			FROM
				DE.VISIT_OCCURRENCE VO ) DB 
		GROUP BY PERSON_ID ) DB
)
SELECT
	(
	SELECT
		COUNT(DISTINCT PERSON_ID)
	FROM
		TBL
	WHERE
		TOT_VISIT_DAYS = MIN) AS MIN_PT_CNT,
	(
	SELECT
		COUNT(DISTINCT PERSON_ID)
	FROM
		TBL
	WHERE
		TOT_VISIT_DAYS = MAX) AS MAX_PT_CNT ;
"""

df_1 = psql.read_sql(query, conn)

In [4]:
df_1

,min_pt_cnt,max_pt_cnt
0,4,1


## [문제 2]

### 문제 2
- 환자들이 진단 받은 상병 내역 중 첫글자는 (a,b,c,d,e) 문자로 시작하고 중간에
“heart” 단어가 포함된 상병 이름을 찾으려고 합니다. condition_occurrence 테이블은
환자가 병원 방문시 진단 받은 질환이 담겨있습니다. 상병코드는
condition_concept_id이고, concept 테이블의 concept_id와 조인하여 상병 이름을
찾을 수 있습니다. (concept_name 컬럼 사용)

    a. 문자 검색시 대소문자를 구분하지 않습니다.     
    b. 상병 이름을 중복없이 나열합니다.

### 문제 2 답안

In [5]:
query = """
SELECT
	DISTINCT CONCEPT_NAME
FROM
	DE.CONDITION_OCCURRENCE
INNER JOIN DE.CONCEPT ON
	CONDITION_CONCEPT_ID = CONCEPT_ID
WHERE
	CONCEPT_NAME ILIKE 'a%heart%'
	OR CONCEPT_NAME ILIKE 'b%heart%'
	OR CONCEPT_NAME ILIKE 'c%heart%'
	OR CONCEPT_NAME ILIKE 'd%heart%'
	OR CONCEPT_NAME ILIKE 'e%heart%' ;
"""

df_2 = psql.read_sql(query, conn)

In [6]:
df_2

,concept_name
0,Chronic congestive heart failure


## [문제 3]

### 문제 3
- drug_exposure 테이블은 환자가 병원에서 처방받은 약의 종류와 처방시작일과
종료일에 대한 정보를 포함하고 있습니다. 환자번호 ‘1891866’ 환자의 약 처방
데이터에서 처방된 약의 종류별로 처음 시작일, 마지막 종료일, 복용일(마지막
종료일과 처음시작일의 차이)을 구하고 복용일이 긴 순으로 정렬하여 테이블을
생성합니다.

    a. 환자번호 : person_id, 약의 종류 : drug_concept_id, 처방시작일 : drug_exposure_start_date, 처방종료일 : drug_exposure_end_date

### 문제 3 답안

In [7]:
query = """
SELECT * FROM DE.DRUG_EXPOSURE
WHERE PERSON_ID = 1891866 ;
"""

df_3 = psql.read_sql(query, conn)

In [8]:
df_3.head()

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,62232837,1891866,19009384,1959-12-01,1959-12-01 18:51:00,1959-12-01,1959-12-01 18:51:00,1959-12-01,38000177,None,...,None,0,0,0,27063979,0,106892,19009384,None,None
1,62232856,1891866,19009384,1965-02-02,1965-02-02 18:51:00,1965-04-20,1965-04-20 18:51:00,1965-04-20,38000177,None,...,None,0,0,0,27063992,0,106892,19009384,None,None
2,62232857,1891866,19009384,1965-04-20,1965-04-20 18:51:00,1965-04-27,1965-04-27 18:51:00,1965-04-27,38000177,None,...,None,0,0,0,27063993,0,106892,19009384,None,None
3,62232858,1891866,19009384,1965-04-27,1965-04-27 18:51:00,1965-11-16,1965-11-16 18:51:00,1965-11-16,38000177,None,...,None,0,0,0,27064141,0,106892,19009384,None,None
4,62232859,1891866,19009384,1965-11-16,1965-11-16 18:51:00,1966-02-15,1966-02-15 18:51:00,1966-02-15,38000177,None,...,None,0,0,0,27063994,0,106892,19009384,None,None


In [9]:
date_cols = ['drug_exposure_start_date', 'drug_exposure_end_date']
df_3[date_cols] = df_3[date_cols].apply(lambda x: pd.to_datetime(x))

In [10]:
tbl_1 = df_3.sort_values(by=['drug_concept_id', 'drug_exposure_start_date'])\
            .groupby('drug_concept_id')\
            .head(1)[['person_id', 'drug_concept_id', 'drug_exposure_start_date']]

In [11]:
tbl_2 = df_3.sort_values(by=['drug_concept_id', 'drug_exposure_end_date'])\
            .groupby('drug_concept_id')\
            .tail(1)[['person_id', 'drug_concept_id', 'drug_exposure_end_date']]

In [12]:
tbl_1.merge(tbl_2, how='inner', on=['person_id', 'drug_concept_id'])\
     .assign(drug_days = lambda x: (x.drug_exposure_end_date - x.drug_exposure_start_date).dt.days)\
     .sort_values(by='drug_days', ascending=False)

,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_end_date,drug_days
1,1891866,19009384,1959-12-01,1998-10-06,14189
2,1891866,19030765,1988-10-18,1998-10-05,3639
3,1891866,40213154,1989-09-12,1998-07-07,3220
0,1891866,1539463,1990-03-13,1998-03-11,2920
4,1891866,40213227,1993-01-05,1993-01-05,0


## [문제 4]

### 문제 4
- drug_exposure 테이블은 환자가 병원에서 처방받은 약의 종류와 처방시작일과
종료일에 대한 정보를 포함하고 있습니다. drug_exposure 테이블로부터 선택된
15가지의 약 번호와 약품명이 저장된 첫번째 drugs 테이블이 있으며, 15가지 약
별로 drug_exposure에 저장된 처방건수가 저장된 두번째 prescription_count
테이블이 있습니다. 마지막으로 drugs 테이블에 해당되는 15가지 약별로 가장 많이
처방되는 약을 짝지어 놓은 drug_pair 테이블이 있습니다. 3개의 테이블을 사용하여
짝지어진 두번째 약의 처방 건수가 첫번째 약의 처방 건수보다 더 많은 첫번째 약의
약품명을 처방건수 순으로 출력합니다.

    a. drugs : drug_concept_id(첫번째약 번호), concept_name(약품명)     
    b. prescription_count : drug_concept_id(첫번째약 번호), cnt(처방건수)      
    c. drug_pair : drug_concept_id1(첫번째약 번호), drug_concept_id2(두번째약 번호)     
    d. 아래 쿼리를 활용하세요.
    
    - with drug_list as (     
select distinct drug_concept_id, concept_name, count(*) as cnt from     
synthea_cdm_1000.drug_exposure de     
join synthea_cdm_1000.concept     
on drug_concept_id = concept_id     
where concept_id in (     
40213154,19078106,19009384,40224172,19127663,1511248,40169216,1539463,     
19126352,1539411,1332419,40163924,19030765,19106768,19075601)     
group by drug_concept_id,concept_name     
order by count(*) desc     
),     
drugs as (select drug_concept_id, concept_name from drug_list)     
, prescription_count as (select drug_concept_id, cnt from drug_list)     

### 문제 4 답안

In [13]:
query = """
select distinct drug_concept_id, concept_name, count(*) as cnt
FROM de.drug_exposure
join de.concept
on drug_concept_id = concept_id
where concept_id in (
40213154,19078106,19009384,40224172,19127663,1511248,40169216,1539463,
19126352,1539411,1332419,40163924,19030765,19106768,19075601)
group by drug_concept_id,concept_name
order by count(*) desc ;
"""

df_4_1 = psql.read_sql(query, conn)
df_4_2 = psql.read_sql("SELECT * FROM DE.DRUG_PAIR ;", conn)

In [14]:
df_4_1.head()

,drug_concept_id,concept_name,cnt
0,40213154,"Influenza, seasonal, injectable, preservative ...",8015
1,19078106,hydrochlorothiazide 25 MG Oral Tablet,3669
2,19009384,"insulin isophane, human 70 UNT/ML / insulin, r...",2825
3,40224172,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...,2814
4,19127663,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...,2729


In [15]:
df_4_2.head()

,drug_concept_id1,drug_concept_id2
0,40213154,19078106
1,19078106,40213154
2,19009384,19030765
3,40224172,40213154
4,19127663,19009384


In [16]:
df_4_2.merge(df_4_1.rename(columns={'drug_concept_id': 'drug_concept_id1',
                                    'concept_name': 'concept_name1',
                                    'cnt': 'cnt1'}),
             how='inner', on='drug_concept_id1')\
      .merge(df_4_1.rename(columns={'drug_concept_id': 'drug_concept_id2',
                                    'concept_name': 'concept_name2',
                                    'cnt': 'cnt2'}),
             how='inner', on='drug_concept_id2')\
      .query('cnt2 > cnt1')\
      .sort_values(by='cnt1')\
      .filter(items=['concept_name1', 'cnt1'])\
      .reset_index(drop=True)

,concept_name1,cnt1
0,clopidogrel 75 MG Oral Tablet,1164
1,hydrochlorothiazide 12.5 MG Oral Tablet,1199
2,1 ML epoetin alfa 4000 UNT/ML Injection [Epogen],1214
3,24 HR metformin hydrochloride 500 MG Extended ...,1235
4,amlodipine 5 MG Oral Tablet,1247
5,simvastatin 20 MG Oral Tablet,1318
6,120 ACTUAT fluticasone propionate 0.044 MG/ACT...,1716
7,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...,2729
8,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...,2814
9,hydrochlorothiazide 25 MG Oral Tablet,3669


## [문제 5]

### 문제 5
- 아래 조건에 모두 해당하는 환자수를 추출합니다.

    a. 제 2형 당뇨병을 진단받은 환자 중에     
i. 당뇨환자의 condition_concept_id 는 다음을 사용합니다.    
3191208,36684827,3194332,3193274,43531010,4130162,45766052,     
45757474,4099651,4129519,4063043,4230254,4193704,4304377,20     
1826,3194082,3192767     

    b. 18세 이상의 환자 중에
    
    c. 진단을 받은 이후 Metformin을 90일 이상 복용한 환자수     
i. drug_concept_id 는 40163924 를 사용합니다.   

### 문제 5 답안

In [17]:
query_1 = """
SELECT
	*
FROM
	(
	SELECT
		CO.*,
		EXTRACT(YEAR FROM AGE(CO.CONDITION_START_DATE, DATE(P.BIRTH_DATETIME))) AS AGE
	FROM
		DE.CONDITION_OCCURRENCE CO
	INNER JOIN DE.PERSON P ON
		CO.PERSON_ID = P.PERSON_ID ) DB
WHERE
	CONDITION_CONCEPT_ID IN (
	3191208, 36684827, 3194332, 3193274, 43531010, 4130162, 45766052, 45757474, 4099651,
	4129519, 4063043, 4230254, 4193704, 4304377, 201826, 3194082, 3192767)
	AND AGE >= 18 ;
"""

df_5_1 = psql.read_sql(query_1, conn)

In [18]:
target_ptids = df_5_1['person_id'].astype('str').values

query_2 = """
SELECT
	*
FROM
	DE.DRUG_EXPOSURE DE
WHERE
	DRUG_CONCEPT_ID = 40163924
    AND PERSON_ID IN ({}); 
""".format(', '.join(target_ptids))

df_5_2 = psql.read_sql(query_2, conn)

In [19]:
tmp = df_5_2.merge(df_5_1[['person_id', 'condition_start_datetime']], how='left', on='person_id')\
            .query('drug_exposure_start_datetime > condition_start_datetime')\
            .assign(metformin_days = lambda x: (x.drug_exposure_end_datetime - x.drug_exposure_start_datetime).dt.days)\
            .groupby('person_id')\
            .agg(tot_metformin_days=('metformin_days', 'sum'))\
            .query('tot_metformin_days >= 90')

In [20]:
tmp

,tot_metformin_days
person_id,
31196,12985
50663,1113
67212,5194
176640,2597
478532,17808
487607,742
495973,3339
510173,7791
531690,9842


In [21]:
tmp.shape[0] # 환자 수

30

## [문제 6]

### 문제 6
- 5.a 항목(제 2형 당뇨병을 진단받은 환자)에서 추출한 환자군의 의약품 처방이
변경된 패턴을 추출하고, 그 빈도의 내림차순으로 나열합니다.

    a. drug_concept_id는 다음을 사용합니다.     
     i. digoxin: 19018935     
     ii. simvastatin: 1539411,1539463     
     iii. clopidogrel: 19075601     
     iv. naproxen: 1115171     
     
    b. 처방 패턴의 예시는 a->b->c 과 같이 나타낼 수 있습니다.     
     i. 예를 들어, a->a->b->c->c 는 위 패턴에 해당합니다. 그러나     
    a->b->a->b->c 는 위 패턴에 해당하지 않습니다.     
     ii. 처방 패턴의 빈도는 a->b->c 와 같은 패턴의 처방을 받은 환자가 몇 명인지로 정의합니다.     
     
    c. 같은 날 처방된 약은 한 그룹으로 묶습니다.     
     i. 괄호가 같은 날의 처방을 나타낸다면, 처방 패턴은 (a, b)->c 와 같이 나타낼 수 있습니다.
     
    d. 데이터에 나타나는 모든 패턴의 빈도를 집계하고, 결과 예시는 다음과 같습니다.
 
| pattern | person_count |
| :--- | :--- |
| (a, b) -> b -> c | 100 |
| a -> b -> c | 90 |
| ... | ... |

### 문제 6 답안

In [22]:
query = """
SELECT
	*,
	(CASE
		WHEN DRUG_CONCEPT_ID = 19018935 THEN 'digoxin'
		WHEN DRUG_CONCEPT_ID IN (1539411, 1539463) THEN 'simvastatin'
		WHEN DRUG_CONCEPT_ID = 19075601 THEN 'clopidogrel'
		WHEN DRUG_CONCEPT_ID = 1115171 THEN 'naproxen'
	END) AS DRUG_FLAG
FROM
	DE.DRUG_EXPOSURE
WHERE
	DRUG_CONCEPT_ID IN (
	19018935, 1539411, 1539463, 19075601, 1115171 )
	AND PERSON_ID IN (
        SELECT
            DISTINCT PERSON_ID
        FROM
            DE.CONDITION_OCCURRENCE CO
        WHERE
            CONDITION_CONCEPT_ID IN (
            3191208, 36684827, 3194332, 3193274, 43531010,
            4130162, 45766052, 45757474, 4099651, 4129519,
            4063043, 4230254, 4193704, 4304377, 201826,
            3194082, 3192767) ) ;
"""

df_6 = psql.read_sql(query, conn)

In [23]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1577 entries, 0 to 1576
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   drug_exposure_id              1577 non-null   int64         
 1   person_id                     1577 non-null   int64         
 2   drug_concept_id               1577 non-null   int64         
 3   drug_exposure_start_date      1577 non-null   object        
 4   drug_exposure_start_datetime  1577 non-null   datetime64[ns]
 5   drug_exposure_end_date        1577 non-null   object        
 6   drug_exposure_end_datetime    1577 non-null   datetime64[ns]
 7   verbatim_end_date             1535 non-null   object        
 8   drug_type_concept_id          1577 non-null   int64         
 9   stop_reason                   0 non-null      object        
 10  refills                       1577 non-null   int64         
 11  quantity                      

In [24]:
df_6.head()

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,drug_flag
0,40931000,31196,1539463,2009-05-11,2009-05-11 17:27:31,2010-05-11,2010-05-11 17:27:31,2010-05-11,38000177,None,...,0,0,0,25713092,0,314231,1539463,None,None,simvastatin
1,40931002,31196,1539463,2010-05-11,2010-05-11 17:27:31,2011-05-11,2011-05-11 17:27:31,2011-05-11,38000177,None,...,0,0,0,25713093,0,314231,1539463,None,None,simvastatin
2,40931004,31196,1539463,2011-05-11,2011-05-11 17:27:31,2012-05-10,2012-05-10 17:27:31,2012-05-10,38000177,None,...,0,0,0,25713094,0,314231,1539463,None,None,simvastatin
3,40931006,31196,1539463,2012-05-10,2012-05-10 17:27:31,2013-05-10,2013-05-10 17:27:31,2013-05-10,38000177,None,...,0,0,0,25713096,0,314231,1539463,None,None,simvastatin
4,40931010,31196,1539463,2013-05-10,2013-05-10 17:27:31,2014-05-10,2014-05-10 17:27:31,2014-05-10,38000177,None,...,0,0,0,25713098,0,314231,1539463,None,None,simvastatin


In [25]:
# 환자가 같은 날 똑같은 의약품을 처방 받은 경우의 중복 제외
# 패턴을 인식할 때, a -> a -> ... 은 a -> ... 이기 때문
tmp_1 = df_6[['person_id', 'drug_flag', 'drug_exposure_start_date']].drop_duplicates()

In [26]:
# 같은 날 처방된 약은 한 그룹으로 묶음
# ()이 묶음 단위임
tmp_2 = \
tmp_1.groupby(['person_id', 'drug_exposure_start_date'])\
     .apply(lambda x: '({})'.format(', '.join(x.drug_flag)))\
     .rename('drug_flag_mod')\
     .to_frame()\
     .reset_index()

In [27]:
tmp_2

,person_id,drug_exposure_start_date,drug_flag_mod
0,31196,1997-05-14,(simvastatin)
1,31196,1998-05-14,(simvastatin)
2,31196,1999-05-14,(simvastatin)
3,31196,2000-05-13,(simvastatin)
4,31196,2001-05-13,(simvastatin)
...,...,...,...
864,2833968,2019-09-09,"(clopidogrel, digoxin, simvastatin)"
865,2833968,2019-10-14,"(clopidogrel, digoxin, simvastatin)"
866,2833968,2019-11-18,"(clopidogrel, digoxin, simvastatin)"
867,2833968,2019-12-09,"(clopidogrel, digoxin, simvastatin)"


In [28]:
# 각 환자마다 받았던 묶음을 리스트에 저장 (pattern 컬럼)
tmp_3 = \
tmp_2.groupby('person_id')\
     .apply(lambda x: list(x.drug_flag_mod))\
     .rename('pattern')\
     .to_frame()\
     .reset_index()

In [29]:
tmp_3.head()

,person_id,pattern
0,31196,"[(simvastatin), (simvastatin), (simvastatin), ..."
1,50663,[(naproxen)]
2,67212,[(naproxen)]
3,170280,"[(naproxen), (naproxen)]"
4,347825,"[(naproxen), (clopidogrel, simvastatin), (clop..."


In [30]:
# 예를 들어 a -> a -> b -> c -> c 인 경우, a -> b -> c로 패턴을 다듬는 함수 정의
def trim_pattern(x):
    tmp_df = pd.DataFrame({'col1': x})
    tmp_df['col2'] = tmp_df['col1'].shift(1)
    tmp_df['flag'] = np.where(tmp_df['col1'] == tmp_df['col2'], np.nan, tmp_df['col1'])
    
    trimed_pattern = ' -> '.join(tmp_df['flag'].dropna().values)
    
    return trimed_pattern

In [31]:
# 패턴 다듬기
tmp_3['pattern'] = tmp_3['pattern'].apply(trim_pattern)

In [32]:
# 다듬어진 패턴 별로 포함되는 환자 수 구하기 및 내림차순 정렬
tmp_4 = \
tmp_3.groupby('pattern')\
     .size()\
     .reset_index(name='person_count')\
     .sort_values('person_count', ascending=False)\
     .reset_index(drop=True)

In [33]:
tmp_4

,pattern,person_count
0,(naproxen),10
1,(simvastatin) -> (naproxen) -> (simvastatin),4
2,(simvastatin),2
3,(simvastatin) -> (clopidogrel),2
4,"(clopidogrel, simvastatin)",1
5,(digoxin) -> (naproxen) -> (digoxin),1
6,"(naproxen) -> (clopidogrel, simvastatin) -> (s...",1
7,"(naproxen) -> (digoxin) -> (digoxin, simvastatin)",1
8,(naproxen) -> (digoxin) -> (simvastatin),1
9,(simvastatin) -> (clopidogrel) -> (simvastatin...,1


In [34]:
tmp_4.to_csv('question_6_answer.csv')

## [문제 7]

## 문제 7
- 제공되는 clinical note 테이블은 한 환자의 의료 기록 샘플입니다.   
note로부터 정보를 추출하여 아래의 4개의 테이블에 입력합니다.   
추출/입력에 사용한 코드와 결과테이블을 제출해야 합니다.   

    a. Regex 또는 Named Entity Recognition 등을 활용하여 제공되는 note에서 정보를 추출     
    i. 제공된 note에서 "CONTINUING" 부분 제외
    
    b. 아래 제공되는 스키마에 테이블 생성      
    i. 테이블 생성 위치는 지원자의 스키마 아래     
    
    c. note 에서 추출한 정보를 테이블에 입력    
    i. sql, python 등 지원자가 익숙한 언어 사용하여 추출     
    ii. 테이블에 입력 시 아래 규칙 적용   
        - 규칙1: 내원일자, 처방일자 >= 환자의 생년월일   
        - 규칙2: 환자 id는 랜덤하게 중복없이 부여    
        - 규칙3: 각 테이블의 id는 랜덤하게 중복없이 부여    
        
        
    d. note에서 정보가 정확히 추출되었는지 검토     
    i. 예: note에서 발견된 질병 키워드의 개수와 condition_occurrence 에 저장된 행 수의 비교    

### 문제 7 답안

In [35]:
query = "SELECT * FROM DE.CLINICAL_NOTE"
df_7 = psql.read_sql(query, conn)

#### A. 제공된 note에서 "CONTINUING" 부분 제외

In [36]:
df_7['trimed_note'] = \
df_7.astype({'note': 'str'})['note'].apply(lambda x: re.split(string=x, pattern=r"[-]*\s*CONTINUING\s*")[0])

In [37]:
print(df_7['trimed_note'][0])

Andrea7 Wolf938
Race:                White
Ethnicity:           Non-Hispanic
Gender:              M
Age:                 55
Birth Date:          1965-04-22
Marital Status:      M
--------------------------------------------------------------------------------
ALLERGIES:
No Known Allergies
--------------------------------------------------------------------------------
ENCOUNTER
2011-06-20 : Encounter at Cape Cod Vet Center : Encounter for Acute bronchitis (disorder)
Type: ambulatory
   
   MEDICATIONS:
  2011-06-20 : Acetaminophen 325 MG Oral Tablet for Acute bronchitis (disorder)
   
   CONDITIONS:
  2011-06-20 : Acute bronchitis (disorder)
   
   CARE PLANS:
  2011-06-20 : Respiratory therapy
                         Reason: Acute bronchitis (disorder)
                         Activity: Recommendation to avoid exercise
                         Activity: Deep breathing and coughing exercises
   
   REPORTS:
   
   OBSERVATIONS:
   
   PROCEDURES:
  2011-06-20 : Sputum examination (pro

#### B. person table 생성
- death_date에 대한 정보를 찾을 수 없으므로, np.nan으로 값을 채움

In [38]:
df_7['birth_date'] = df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                                                   pattern=r"Birth Date[:]\s*(?P<birth>\d*[-]\d*[-]\d*)\s*")\
                                                           .group('birth'))

In [39]:
df_7['birth_date'] = pd.to_datetime(df_7['birth_date'])
df_7 = df_7.assign(year_of_birth = lambda x: x.birth_date.dt.year,
                   month_of_birth = lambda x: x.birth_date.dt.month,
                   day_of_birth = lambda x: x.birth_date.dt.day)

In [40]:
df_7['gender_value'] = df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                                                     pattern=r"Gender[:]\s*(?P<gender>\w*)\s*")\
                                                             .group('gender'))

In [41]:
df_7['race_value'] = df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                                                   pattern=r"Race[:]\s*(?P<race>\w*)\s*")\
                                                           .group('race'))

In [42]:
df_7['ethnicity_value'] = df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                                                        pattern=r"Ethnicity[:]\s*(?P<ethnicity>.*)\s*")\
                                                                .group('ethnicity'))

In [43]:
df_7 = df_7.assign(person_id = np.random.choice(len(df_7), len(df_7), replace=False) + 1,
                   death_date = np.nan)

person = df_7.filter(items=['person_id', 'year_of_birth', 'month_of_birth', 'day_of_birth',
                            'death_date', 'gender_value', 'race_value', 'ethnicity_value'])

In [44]:
person.head()

,person_id,year_of_birth,month_of_birth,day_of_birth,death_date,gender_value,race_value,ethnicity_value
0,5,1965,4,22,NaN,M,White,Non-Hispanic
1,19,1965,4,22,NaN,M,White,Non-Hispanic
2,2,1965,4,22,NaN,M,White,Non-Hispanic
3,17,1965,4,22,NaN,M,White,Non-Hispanic
4,12,1965,4,22,NaN,M,White,Non-Hispanic


#### C. visit_occurrence table 생성

In [45]:
df_7['visit_start_date'] = \
df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                              pattern=r"ENCOUNTER\s*(?P<visit_date>\d*[-]\d*[-]\d*)\s*[:]\s*Encounter at")\
                                      .group('visit_date'))

df_7['visit_start_date'] = pd.to_datetime(df_7['visit_start_date'])

In [46]:
df_7['care_site_nm'] = \
df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                              pattern=r"Encounter at\s*(?P<care_site>[\w*\s*]*\w+)\s")\
                                      .group('care_site'))

In [47]:
df_7['visit_type_value'] = \
df_7['trimed_note'].apply(lambda x: re.search(string=x,
                                              pattern=r"Type[:]\s*(?P<type>.*\w+)\s*")\
                                      .group('type'))

In [48]:
df_7 = df_7.assign(visit_occurrence_id = np.random.choice(len(df_7), len(df_7), replace=False) + 1)

In [49]:
visit_occurrence = df_7.filter(items=['visit_occurrence_id', 'person_id', 'visit_start_date',
                                      'care_site_nm', 'visit_type_value'])

- 규칙1: 내원일자 >= 환자의 생년월일

In [50]:
tmp = \
person.rename(columns = {'year_of_birth': 'year', 'month_of_birth': 'month', 'day_of_birth': 'day'})\
      .assign(birth_date = lambda x: pd.to_datetime(x[['year', 'month', 'day']]))\
      .filter(items=['person_id', 'birth_date'])

In [51]:
visit_occurrence = \
visit_occurrence.merge(tmp, how='left', on='person_id')\
                .query("visit_start_date >= birth_date")\
                .drop(['birth_date'], axis=1)

In [52]:
visit_occurrence.head()

,visit_occurrence_id,person_id,visit_start_date,care_site_nm,visit_type_value
0,8,5,2011-06-20,Cape Cod Vet Center,ambulatory
1,16,19,2015-04-23,Cape Cod Vet Center,ambulatory
2,14,2,2015-04-23,Cape Cod Vet Center,ambulatory
3,6,17,2013-05-16,Cape Cod Vet Center,wellness
4,20,12,2011-05-12,Cape Cod Vet Center,wellness


#### d. drug_exposure table 생성

In [128]:
emp_list = list()

for pt_id, vo_id, note in df_7[['person_id', 'visit_occurrence_id', 'trimed_note']].values:

    note = str(note)
    note = re.split(pattern=r"\s*MEDICATIONS:", string=note)[1]
    note = re.split(pattern=r"\sCONDITIONS:", string=note)[0]

    pattern = re.compile(r"\s*(?P<date>\d*[-]\d*[-]\d*)\s*[:]\s*(?P<drug>\w+)\s*(?P<dose>\d+)\s*(?P<unit>\w+)\s*(?P<route>\w+)")

    result = pd.DataFrame(pattern.findall(note), columns=['drug_exposure_start_date', 'drug_value',
                                                          'dose_value', 'unit_value', 'route_value'])
    
    result = result.assign(person_id = pt_id, visit_occurrence_id = vo_id)
    
    emp_list.append(result)

In [129]:
tmp_df = pd.concat(emp_list)
drug_exposure = tmp_df.assign(drug_exposure_id = np.random.choice(len(tmp_df), len(tmp_df), replace=False) + 1)\
                      .filter(items=['drug_exposure_id', 'person_id',
                                     'drug_exposure_start_date', 'drug_value',
                                     'route_value', 'dose_value', 'unit_value', 'visit_occurrence_id'])\
                      .reset_index(drop=True)

- 규칙1: 처방일자 >= 환자의 생년월일

In [130]:
drug_exposure['drug_exposure_start_date'] = pd.to_datetime(drug_exposure['drug_exposure_start_date'])

In [131]:
tmp = \
person.rename(columns = {'year_of_birth': 'year', 'month_of_birth': 'month', 'day_of_birth': 'day'})\
      .assign(birth_date = lambda x: pd.to_datetime(x[['year', 'month', 'day']]))\
      .filter(items=['person_id', 'birth_date'])

In [132]:
drug_exposure = \
drug_exposure.merge(tmp, how='left', on='person_id')\
             .query("drug_exposure_start_date >= birth_date")\
             .drop(['birth_date'], axis=1)

In [133]:
drug_exposure

,drug_exposure_id,person_id,drug_exposure_start_date,drug_value,route_value,dose_value,unit_value,visit_occurrence_id
0,1,5,2011-06-20,Acetaminophen,Oral,325,MG,8
1,2,21,1984-06-07,FLUoxetine,Oral,20,MG,4


#### e. condition_occurrence table 생성

In [59]:
emp_list = list()

for pt_id, vo_id, note in df_7[['person_id', 'visit_occurrence_id', 'trimed_note']].values:
    
    note = str(note)
    
    note = re.split(pattern=r"\s*CONDITIONS:", string=note)[1]
    note = re.split(pattern=r"\sCARE PLANS:", string=note)[0]
    
    pattern = re.compile(r"\s*(?P<date>\d*[-]\d*[-]\d*)\s*[:](?P<condition>\s*.*)")
    result = pd.DataFrame(pattern.findall(note), columns=['condition_start_date', 'condition_value'])
    result = result.assign(person_id = pt_id, visit_occurrence_id = vo_id)
    
    emp_list.append(result)

In [60]:
tmp_df = pd.concat(emp_list)
condition_occurrence = tmp_df.assign(condition_occurrence_id = np.random.choice(len(tmp_df), len(tmp_df), replace=False) + 1)\
                             .filter(items=['condition_occurrence_id', 'person_id', 'condition_start_date',
                                            'condition_value', 'visit_occurrence_id'])\
                             .reset_index(drop=True)

In [61]:
condition_occurrence['condition_start_date'] = pd.to_datetime(condition_occurrence['condition_start_date'])

In [62]:
condition_occurrence

,condition_occurrence_id,person_id,condition_start_date,condition_value,visit_occurrence_id
0,16,5,2011-06-20,Acute bronchitis (disorder),8
1,5,19,2015-04-23,Anemia (disorder),16
2,2,17,2013-05-16,Prediabetes,6
3,10,16,1977-04-10,Chronic sinusitis (disorder),11
4,3,9,1996-07-04,Body mass index 30+ - obesity (finding),7
5,13,21,1984-06-07,Major depression disorder,4
6,14,22,2019-04-21,Acute bacterial sinusitis (disorder),19
7,15,20,2014-01-26,Viral sinusitis (disorder),3
8,12,23,2020-02-25,COVID-19,23
9,9,23,2020-02-25,Suspected COVID-19,23


#### f. Create tables and insert data

- person table

In [146]:
person

,person_id,year_of_birth,month_of_birth,day_of_birth,death_date,gender_value,race_value,ethnicity_value
0,5,1965,4,22,NaN,M,White,Non-Hispanic
1,19,1965,4,22,NaN,M,White,Non-Hispanic
2,2,1965,4,22,NaN,M,White,Non-Hispanic
3,17,1965,4,22,NaN,M,White,Non-Hispanic
4,12,1965,4,22,NaN,M,White,Non-Hispanic
5,16,1965,4,22,NaN,M,White,Non-Hispanic
6,4,1965,4,22,NaN,M,White,Non-Hispanic
7,7,1965,4,22,NaN,M,White,Non-Hispanic
8,9,1965,4,22,NaN,M,White,Non-Hispanic
9,1,1965,4,22,NaN,M,White,Non-Hispanic


In [100]:
with open('DBMS_config.json', 'r') as fp:
    dbms_params = json.load(fp)
    conn = psycopg2.connect(**dbms_params)

In [101]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS walker101.person")

create_p_table_query = """
CREATE TABLE walker101.person (
    person_id BIGINT PRIMARY KEY,
    year_of_birth INTEGER NOT NULL,
    month_of_birth INTEGER,
    day_of_birth INTEGER,
    death_date TIMESTAMP,
    gender_value VARCHAR(50),
    race_value VARCHAR(50),
    ethnicity_value VARCHAR(50)
)
"""
cursor.execute(create_p_table_query)

print("Table created successfully...")
conn.commit()

Table created successfully...


In [102]:
p_table_insert_string = \
', '.join([str(person.to_records(index=False)[i]).replace('nan', 'null') for i in range(len(person))])

cursor = conn.cursor()

p_table_insert_query = """
INSERT INTO walker101.person(person_id, year_of_birth, month_of_birth, day_of_birth, death_date,
                             gender_value, race_value, ethnicity_value)
values {}
""".format(p_table_insert_string)

cursor.execute(p_table_insert_query)

print("Data inserted successfully...")
conn.commit()
conn.close()

Data inserted successfully...


- visit_occurrence table

In [103]:
visit_occurrence

,visit_occurrence_id,person_id,visit_start_date,care_site_nm,visit_type_value
0,8,5,2011-06-20,Cape Cod Vet Center,ambulatory
1,16,19,2015-04-23,Cape Cod Vet Center,ambulatory
2,14,2,2015-04-23,Cape Cod Vet Center,ambulatory
3,6,17,2013-05-16,Cape Cod Vet Center,wellness
4,20,12,2011-05-12,Cape Cod Vet Center,wellness
5,11,16,1977-04-10,FALMOUTH HOSPITAL,ambulatory
6,5,4,2017-05-04,Cape Cod Vet Center,wellness
7,13,7,2016-04-28,Cape Cod Vet Center,wellness
8,7,9,1996-07-04,Cape Cod Vet Center,wellness
9,22,1,2019-05-16,Cape Cod Vet Center,wellness


In [112]:
with open('DBMS_config.json', 'r') as fp:
    dbms_params = json.load(fp)
    conn = psycopg2.connect(**dbms_params)

In [113]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS walker101.visit_occurrence")

create_vo_table_query = """
CREATE TABLE walker101.visit_occurrence (
    visit_occurrence_id BIGINT PRIMARY KEY,
    person_id BIGINT NOT NULL,
    visit_start_date DATE,
    care_site_nm TEXT,
    visit_type_value VARCHAR(50),
    FOREIGN KEY (person_id)
    REFERENCES walker101.person (person_id)
    ON UPDATE CASCADE ON DELETE CASCADE
)
"""
cursor.execute(create_vo_table_query)

print("Table created successfully...")
conn.commit()

Table created successfully...


In [114]:
vo_table_insert_string = \
', '.join([str(visit_occurrence.to_records(index=False)[i]).replace('nan', 'null') for i in range(len(visit_occurrence))])

cursor = conn.cursor()

vo_table_insert_query = """
INSERT INTO walker101.visit_occurrence(visit_occurrence_id, person_id, visit_start_date, care_site_nm, visit_type_value)
values {}
""".format(vo_table_insert_string)

cursor.execute(vo_table_insert_query)

print("Data inserted successfully...")
conn.commit()
conn.close()

Data inserted successfully...


- drug_exposure table

In [115]:
drug_exposure

,drug_exposure_id,person_id,drug_exposure_start_date,drug_value,route_value,dose_value,unit_value
0,1,5,2011-06-20,Acetaminophen,Oral,325,MG
1,2,21,1984-06-07,FLUoxetine,Oral,20,MG


In [135]:
with open('DBMS_config.json', 'r') as fp:
    dbms_params = json.load(fp)
    conn = psycopg2.connect(**dbms_params)

In [136]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS walker101.drug_exposure")

create_de_table_query = """
CREATE TABLE walker101.drug_exposure (
    drug_exposure_id BIGINT PRIMARY KEY,
    person_id BIGINT NOT NULL,
    drug_exposure_start_date DATE NOT NULL,
    drug_value TEXT,
    route_value VARCHAR(50),
    dose_value VARCHAR(50),
    unit_value VARCHAR(50),
    visit_occurrence_id BIGINT,
    
    FOREIGN KEY (person_id)
        REFERENCES walker101.person (person_id)
        ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY (visit_occurrence_id)
        REFERENCES walker101.visit_occurrence (visit_occurrence_id)
        ON UPDATE CASCADE ON DELETE CASCADE
)
"""
cursor.execute(create_de_table_query)

print("Table created successfully...")
conn.commit()

Table created successfully...


In [137]:
de_table_insert_string = \
', '.join([str(drug_exposure.to_records(index=False)[i]).replace('nan', 'null') for i in range(len(drug_exposure))])

cursor = conn.cursor()

de_table_insert_query = """
INSERT INTO walker101.drug_exposure(drug_exposure_id, person_id, drug_exposure_start_date,
                                    drug_value, route_value, dose_value, unit_value, visit_occurrence_id)
values {}
""".format(de_table_insert_string)

cursor.execute(de_table_insert_query)

print("Data inserted successfully...")
conn.commit()
conn.close()

Data inserted successfully...


- condition_occurrence table

In [138]:
condition_occurrence

,condition_occurrence_id,person_id,condition_start_date,condition_value,visit_occurrence_id
0,16,5,2011-06-20,Acute bronchitis (disorder),8
1,5,19,2015-04-23,Anemia (disorder),16
2,2,17,2013-05-16,Prediabetes,6
3,10,16,1977-04-10,Chronic sinusitis (disorder),11
4,3,9,1996-07-04,Body mass index 30+ - obesity (finding),7
5,13,21,1984-06-07,Major depression disorder,4
6,14,22,2019-04-21,Acute bacterial sinusitis (disorder),19
7,15,20,2014-01-26,Viral sinusitis (disorder),3
8,12,23,2020-02-25,COVID-19,23
9,9,23,2020-02-25,Suspected COVID-19,23


In [143]:
with open('DBMS_config.json', 'r') as fp:
    dbms_params = json.load(fp)
    conn = psycopg2.connect(**dbms_params)

In [144]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS walker101.condition_occurrence")

create_co_table_query = """
CREATE TABLE walker101.condition_occurrence (
    condition_occurrence_id BIGINT PRIMARY KEY,
    person_id BIGINT NOT NULL,
    condition_start_date DATE NOT NULL,
    condition_value TEXT,
    visit_occurrence_id BIGINT,
    
    FOREIGN KEY (person_id)
        REFERENCES walker101.person (person_id)
        ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY (visit_occurrence_id)
        REFERENCES walker101.visit_occurrence (visit_occurrence_id)
        ON UPDATE CASCADE ON DELETE CASCADE
)
"""
cursor.execute(create_co_table_query)

print("Table created successfully...")
conn.commit()

Table created successfully...


In [145]:
co_table_insert_string = \
', '.join([str(condition_occurrence.to_records(index=False)[i]).replace('nan', 'null') for i in range(len(condition_occurrence))])

cursor = conn.cursor()

co_table_insert_query = """
INSERT INTO walker101.condition_occurrence(condition_occurrence_id, person_id, condition_start_date, condition_value, visit_occurrence_id)
values {}
""".format(co_table_insert_string)

cursor.execute(co_table_insert_query)

print("Data inserted successfully...")
conn.commit()
conn.close()

Data inserted successfully...
